In [125]:
import string
import random
import itertools
from cipher.keyword_cipher import *
from support.utilities import *

In [11]:
pt = "catch the cat"

ca = list(string.ascii_lowercase)
random.shuffle(ca)
ca = cat(ca)
ca

'gpavtdyzocqnrsujmxikwbehlf'

In [18]:
ct = keyword_encipher(pt, ca)
ct

'agkaz kzt agk'

In [104]:
def show_mapping_alpha(c_a, p_a=string.ascii_lowercase, letters=string.ascii_lowercase):
    mapping = {p: c for (p, c) in zip(p_a, c_a) if p in letters}
    return show_mapping(mapping)

def show_mapping(mapping):
    retval  = '| plaintext letter  | ' + ' | '.join(l for l in sorted(mapping)) + ' |\n'
    retval += '|-------------------|---|---|---|---|---|\n'
    retval += '| ciphertext letter | ' + ' | '.join(mapping[l] for l in sorted(mapping)) + ' |\n'
    return retval

In [105]:
print(show_mapping_alpha(ca, letters=sanitise(pt)))

| plaintext letter  | a | c | e | h | t |
|-------------------|---|---|---|---|---|
| ciphertext letter | g | a | t | z | k |



In [17]:
m0 = {p: c for (p, c) in zip(string.ascii_letters, ca) if p in pt}
im0 = {c: p for (p, c) in zip(string.ascii_letters, ca) if p in pt}
m0, im0

({'a': 'g', 'c': 'a', 'e': 't', 'h': 'z', 't': 'k'},
 {'g': 'a', 'a': 'c', 't': 'e', 'z': 'h', 'k': 't'})

In [55]:
print(show_mapping(m0))

| a | c | e | h | t |
| g | a | t | z | k |



In [61]:
def apply_inverse_map(ciphertext, mapping):
    plaintext = cat(mapping[l] if l in mapping else l for l in ciphertext)
    return plaintext, Pbigrams(sanitise(plaintext))

In [44]:
def swap(letters, i, j):
    if i > j:
        i, j = j, i
    if i == j:
        return letters
    else:
        return (letters[:i] + letters[j] + letters[i+1:j] + letters[i] +
                letters[j+1:])

In [50]:
def map_swap(mapping):
    keys = sorted(mapping)
    values = cat(mapping[l] for l in keys)
    n = len(keys)
    swapped_values = swap(values, random.randrange(n), random.randrange(n))
    return {k: sv for (k, sv) in zip(keys, swapped_values)}

In [89]:
im1 = map_swap(im0)
im2 = map_swap(im1)

In [90]:
apply_inverse_map(ct, im2)

('aceah eht ace', -24.470656262279007)

In [91]:
apply_inverse_map(ct, im1)

('actah the act', -23.337953804339712)

In [88]:
apply_inverse_map(ct, im0)

('catch the cat', -22.142275954584633)

In [92]:
im0, im1, im2

({'g': 'a', 'a': 'c', 't': 'e', 'z': 'h', 'k': 't'},
 {'a': 'a', 'g': 'c', 'k': 't', 't': 'e', 'z': 'h'},
 {'a': 'a', 'g': 'c', 'k': 'e', 't': 't', 'z': 'h'})

In [93]:
m1 = {im1[l]: l for l in im1}
m2 = {im2[l]: l for l in im2}

In [106]:
print(show_mapping(m0))

| plaintext letter  | a | c | e | h | t |
|-------------------|---|---|---|---|---|
| ciphertext letter | g | a | t | z | k |



In [107]:
print(show_mapping(m1))

| plaintext letter  | a | c | e | h | t |
|-------------------|---|---|---|---|---|
| ciphertext letter | a | g | t | z | k |



In [108]:
print(show_mapping(m2))

| plaintext letter  | a | c | e | h | t |
|-------------------|---|---|---|---|---|
| ciphertext letter | a | g | k | z | t |



In [110]:
im3 = map_swap(im2)
apply_inverse_map(ct, im3)

('hceha eat hce', -26.41716766077668)

In [111]:
m3 = {im3[l]: l for l in im3}

In [112]:
print(show_mapping(m3))

| plaintext letter  | a | c | e | h | t |
|-------------------|---|---|---|---|---|
| ciphertext letter | z | g | k | a | t |



In [113]:
def all_swaps(mapping):
    keys = sorted(mapping)
    values = cat(mapping[l] for l in keys)
    n = len(keys)
    swapped_values = [swap(values, i, j) for i in range(n) for j in range(n) if i < j]
    return [{k: sv for (k, sv) in zip(keys, svs)} for svs in swapped_values]

In [117]:
im0, [apply_inverse_map(ct, tim) for tim in all_swaps(im0)]

({'g': 'a', 'a': 'c', 't': 'e', 'z': 'h', 'k': 't'},
 [('actah the act', -23.337953804339712),
  ('tacth che tac', -22.992889593694795),
  ('eateh thc eat', -23.337174988961543),
  ('hathc tce hat', -24.20565798548872),
  ('ctach ahe cta', -23.361982341471602),
  ('cetch tha cet', -23.152196785128968),
  ('chtca tae cht', -25.47053856384374),
  ('caech eht cae', -27.119008761052356),
  ('cahct hte cah', -25.96020844569102),
  ('catce teh cat', -24.369461369323975)])

In [118]:
im2, [apply_inverse_map(ct, tim) for tim in all_swaps(im2)]

({'a': 'a', 'g': 'c', 'k': 'e', 't': 't', 'z': 'h'},
 [('caech eht cae', -27.119008761052356),
  ('ecaeh aht eca', -26.10317913928645),
  ('tceth eha tce', -23.289877585658743),
  ('hceha eat hce', -26.41716766077668),
  ('aecah cht aec', -28.466074945814817),
  ('ateah ehc ate', -23.89678491033435),
  ('aheac ect ahe', -23.82052347276842),
  ('actah the act', -23.337953804339712),
  ('achae het ach', -24.4061387567535),
  ('aceat eth ace', -21.139211036323402)])

In [119]:
def all_swaps_worse(mapping):
    _, score0 = apply_inverse_map(ct, mapping)
    swapped_mappings = all_swaps(mapping)
    scores = [apply_inverse_map(ct, m)[1] for m in swapped_mappings]
    better_scores = [s for s in scores if s > score0]
    return better_scores == []

In [123]:
all_swaps_worse(im3)

False

In [124]:
def make_map(als, bls):
    return {a: b for (a, b) in zip(als, bls)}

In [129]:
ptls = cat(sorted(deduplicate(sanitise(pt))))
ctls = cat(sorted(deduplicate(sanitise(ct))))
ptls, ctls

('aceht', 'agktz')

In [132]:
all_maps = [make_map(c, ptls) for c in itertools.permutations(ctls)]
len(all_maps), all_maps[:5]

(120,
 [{'a': 'a', 'g': 'c', 'k': 'e', 't': 'h', 'z': 't'},
  {'a': 'a', 'g': 'c', 'k': 'e', 'z': 'h', 't': 't'},
  {'a': 'a', 'g': 'c', 't': 'e', 'k': 'h', 'z': 't'},
  {'a': 'a', 'g': 'c', 't': 'e', 'z': 'h', 'k': 't'},
  {'a': 'a', 'g': 'c', 'z': 'e', 'k': 'h', 't': 't'}])

In [138]:
local_optima = [m for m in all_maps if all_swaps_worse(m) if m != im0]
local_optima

[{'a': 'a', 'z': 'c', 'k': 'e', 't': 'h', 'g': 't'},
 {'g': 'a', 't': 'c', 'z': 'e', 'a': 'h', 'k': 't'},
 {'g': 'a', 'z': 'c', 'a': 'e', 't': 'h', 'k': 't'},
 {'t': 'a', 'k': 'c', 'g': 'e', 'z': 'h', 'a': 't'},
 {'t': 'a', 'z': 'c', 'k': 'e', 'g': 'h', 'a': 't'},
 {'z': 'a', 't': 'c', 'a': 'e', 'k': 'h', 'g': 't'}]

In [143]:
apply_inverse_map(ct, local_optima[3]), [apply_inverse_map(ct, tim) for tim in all_swaps(local_optima[3])]

(('tecth cha tec', -22.37718617528681),
 [('etceh cha etc', -27.45222919076422),
  ('cetch tha cet', -23.152196785128968),
  ('aecah cht aec', -28.466074945814817),
  ('hecht cta hec', -24.0528877258752),
  ('tceth eha tce', -23.289877585658743),
  ('tacth che tac', -22.992889593694795),
  ('thcte cea thc', -23.37530629522044),
  ('teath ahc tea', -23.192822966291835),
  ('tehtc hca teh', -25.824045558109102),
  ('tecta cah tec', -23.630623398955464)])

In [145]:
apply_inverse_map(ct, local_optima[5]), [apply_inverse_map(ct, tim) for tim in all_swaps(local_optima[5])]

(('ethea hac eth', -21.831799648932474),
 [('tehta hac teh', -25.2630658238322),
  ('hteha eac hte', -25.12161519433393),
  ('cthca hae cth', -25.56645047924706),
  ('athae hec ath', -22.523920547555058),
  ('ehtea tac eht', -24.414224893001006),
  ('echea hat ech', -22.34614937355321),
  ('eahet htc eah', -24.64789885786501),
  ('etcea cah etc', -24.40643936994998),
  ('etaeh ahc eta', -27.042650227267693),
  ('ethec hca eth', -23.70218668022281)])

In [140]:
apply_inverse_map(ct, im0)

('catch the cat', -22.142275954584633)

In [146]:
local_optima[3]

{'t': 'a', 'k': 'c', 'g': 'e', 'z': 'h', 'a': 't'}

In [147]:
l3 = {local_optima[3][l]: l for l in local_optima[3]}
print(show_mapping(l3))

| plaintext letter  | a | c | e | h | t |
|-------------------|---|---|---|---|---|
| ciphertext letter | t | k | g | z | a |



In [150]:
cat(p[0] for p in english_counts.most_common())

'etoainhsrdlumwycfgpbvkxjqz'